In [27]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import psycopg2


In [28]:
def fetch_candles(product_id, start, end, granularity):
    url = f"https://api.pro.coinbase.com/products/{product_id}/candles"
    params = {
        'start': start,
        'end': end,
        'granularity': granularity
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch data: {response.status_code} {response.text}")

In [29]:
def append_to_postgres(cur, time, low, high, open, close, volume):
    cur.execute("INSERT INTO eth_2022_2023(time, low, high, open, close, volume) VALUES (%s, %s, %s, %s, %s, %s)",
                (time, low, high, open, close, volume))

In [30]:
# Database configuration
conn = psycopg2.connect(
    database="ethereum", 
    user="postgres", 
    password="PASSWORD", 
    host="localhost", 
    port="5432"
)
cur = conn.cursor()

# API configuration
product_id = "ETH-USD"
start_date = datetime(2022, 1, 1)  # starting from the beginning of the year
end_date = datetime(2023, 1, 1)  # up to the current time
granularity = 300  # Daily granularity

# Loop through each day and fetch data
current_date = start_date
while current_date < end_date:
    next_date = current_date + timedelta(days=1)
    try:
        daily_data = fetch_candles(product_id, current_date.isoformat() + "Z", next_date.isoformat() + "Z", granularity)
        for entry in daily_data:
            time = datetime.fromtimestamp(entry[0]).isoformat()
            low = entry[1]
            high = entry[2]
            open = entry[3]
            close = entry[4]
            volume = entry[5]
            append_to_postgres(cur, time, low, high, open, close, volume)
        print(f"Fetched and inserted data for {current_date.date()} to {next_date.date()}")
    except Exception as e:
        print(e)
    current_date = next_date

# Commit changes and close connection
conn.commit()
cur.close()
conn.close()

In [31]:
def save_to_csv(data, filename):
    df = pd.DataFrame(data, columns=['time', 'low', 'high', 'open', 'close', 'volume'])
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.sort_values('time', ascending=True, inplace=True)  # Ensure data is in chronological order
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

In [32]:
all_data = []
# Loop through each day and fetch data
current_date = start_date
while current_date < end_date:
    next_date = current_date + timedelta(days=1)
    try:
        daily_data = fetch_candles(product_id, current_date.isoformat() + "Z", next_date.isoformat() + "Z", granularity)
        all_data.extend(daily_data)
        print(f"Fetched data for {current_date.date()} to {next_date.date()}")
    except Exception as e:
        print(e)
    current_date = next_date

# Save all fetched data to CSV
save_to_csv(all_data, 'eth_2022_2023.csv')

Fetched data for 2022-01-01 to 2022-01-02
Fetched data for 2022-01-02 to 2022-01-03
Fetched data for 2022-01-03 to 2022-01-04
Fetched data for 2022-01-04 to 2022-01-05
Fetched data for 2022-01-05 to 2022-01-06
Fetched data for 2022-01-06 to 2022-01-07
Fetched data for 2022-01-07 to 2022-01-08
Fetched data for 2022-01-08 to 2022-01-09
Fetched data for 2022-01-09 to 2022-01-10
Fetched data for 2022-01-10 to 2022-01-11
Fetched data for 2022-01-11 to 2022-01-12
Fetched data for 2022-01-12 to 2022-01-13
Fetched data for 2022-01-13 to 2022-01-14
Fetched data for 2022-01-14 to 2022-01-15
Fetched data for 2022-01-15 to 2022-01-16
Fetched data for 2022-01-16 to 2022-01-17
Fetched data for 2022-01-17 to 2022-01-18
Fetched data for 2022-01-18 to 2022-01-19
Fetched data for 2022-01-19 to 2022-01-20
Fetched data for 2022-01-20 to 2022-01-21
Fetched data for 2022-01-21 to 2022-01-22
Fetched data for 2022-01-22 to 2022-01-23
Fetched data for 2022-01-23 to 2022-01-24
Fetched data for 2022-01-24 to 202